# <font color='Black'>Scraping Neighborhood information from wikipedia</font>

#### I import the usefull libraries and directly create a dataframe from the table of the wikipedia page :

In [122]:
import pandas as pd
import wikipedia as wp

html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### I deal with information that we doesn't have by excluding boroughs equal to "Not assigned" :

In [123]:
df_filtered = df[df['Borough'] != 'Not assigned']

In [124]:
df_filtered.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [125]:
df_filtered.loc[df['Postal Code'] == 'M5A']

,Postal Code,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Finally, i get the number of rows and columns from the dataframe that I created :

In [126]:
df_filtered.shape

(103, 3)

#### I import geospatial data from the csv file

In [127]:
geoloc = pd.read_csv('http://cocl.us/Geospatial_data')

In [128]:
geoloc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


#### I merge the 2 dataframes to get a new filtered one with information on latitude and longitude for each Postal Code

In [129]:
df_filtered2 = pd.merge(df_filtered, geoloc, on='Postal Code')
df_filtered2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [130]:
df_filtered2.loc[df['Postal Code'] == 'M1B']

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### I import every libraries that will be useful in the next section

In [131]:
import folium
import requests
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

#### I create a map of Toronto by neighborhoods 

In [132]:
map_toronto = folium.Map(location=[43.753259, -79.329656], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_filtered2['Latitude'], df_filtered2['Longitude'], df_filtered2['Borough'], df_filtered2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

#### I use the KMeans method to create clusters from the data I got on Toronto's neighborhoods

In [134]:
kclusters = 5

toronto_grouped_clustering = df_filtered2.drop(['Postal Code','Borough','Neighborhood'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([4, 4, 2, 3, 2, 1, 0, 4, 4, 2], dtype=int32)

#### I insert a new column with the cluster labels I got from the preview cell

In [135]:
df_filtered2.insert(0, 'Cluster Labels', kmeans.labels_)

In [136]:
df_filtered2

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,1,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,4,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,1,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### I create a new map but with a color for every different cluster I got

In [138]:
map_toronto = folium.Map(location=[43.6532,-79.3832],zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat,lng,borough,neighborhood,cluster in zip(df_filtered2['Latitude'],df_filtered2['Longitude'],df_filtered2['Borough'],df_filtered2['Neighborhood'],df_filtered2['Cluster Labels']):
    #label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup('Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.75,
    parse_html=False).add_to(map_toronto)
    
map_toronto